In [19]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as Func

import matplotlib.pyplot as plt

from tqdm import tqdm

In [20]:
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [21]:
path='./Iris.csv'
fullData=pd.read_csv(path)
fullData=fullData.sample(frac=1)
fullData[0:4]

# dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True, num_workers = 4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
139,140,6.9,3.1,5.4,2.1,Iris-virginica
100,101,6.3,3.3,6.0,2.5,Iris-virginica
138,139,6.0,3.0,4.8,1.8,Iris-virginica
8,9,4.4,2.9,1.4,0.2,Iris-setosa


In [22]:
fullData=fullData.drop(columns='Id')
clases=fullData.iloc[:,4].unique()
print(clases)

['Iris-virginica' 'Iris-setosa' 'Iris-versicolor']


In [23]:
newLabel={
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
fullData.replace(newLabel,inplace=True)

In [24]:
fullData

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
139,6.9,3.1,5.4,2.1,2
100,6.3,3.3,6.0,2.5,2
138,6.0,3.0,4.8,1.8,2
8,4.4,2.9,1.4,0.2,0
58,6.6,2.9,4.6,1.3,1
...,...,...,...,...,...
126,6.2,2.8,4.8,1.8,2
104,6.5,3.0,5.8,2.2,2
3,4.6,3.1,1.5,0.2,0
86,6.7,3.1,4.7,1.5,1


In [25]:

CountOfRows=fullData.shape[0]
testData=fullData[0:CountOfRows//3].reset_index(drop=True)
trainData=fullData[CountOfRows//3:CountOfRows].reset_index(drop=True)
trainData


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,6.1,3.0,4.9,1.8,2
1,6.7,2.5,5.8,1.8,2
2,5.8,2.6,4.0,1.2,1
3,5.0,3.4,1.5,0.2,0
4,4.9,3.0,1.4,0.2,0
...,...,...,...,...,...
95,6.2,2.8,4.8,1.8,2
96,6.5,3.0,5.8,2.2,2
97,4.6,3.1,1.5,0.2,0
98,6.7,3.1,4.7,1.5,1


In [26]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
    return in_file

In [27]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,csv_file):
        data=csv_file
        x=data.iloc[:,:4]
        x=normalizeCollumns(x)
        y=data.iloc[:,4]
        self.x_train=torch.Tensor(np.array(x))
        self.y_train=nn.functional.one_hot(torch.Tensor(np.array(y)).to(torch.int64),num_classes=3)
        
        
    def __len__(self):
        return len(self.y_train)
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [28]:
trainDataset=MyDataset(trainData)
trainDataloader= torch.utils.data.DataLoader(trainDataset,batch_size=5)

In [29]:
class IrisClassifier(torch.nn.Module):
    def __init__(self,countOfParametrs=4):
        super().__init__()
        act_func=torch.nn.ReLU()
        linear_1 = torch.nn.Linear(countOfParametrs,4*countOfParametrs)
        linear_2= torch.nn.Linear(4*countOfParametrs,3)

        self.model = torch.nn.Sequential(
            linear_1,act_func,linear_2
        )

    def forward(self,x):
        return self.model(x)

In [30]:
model=IrisClassifier().to(device)
loss_func=torch.nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=1e-2)

In [31]:
def accruracy(pred,label):
    answer = pred.detach().numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()


In [32]:
def trainModel(dataloader,model,loss_fn,optimiser):
    loss_val=0
    accruracy_val=0
    for sample in tqdm(dataloader):
        X,Y=sample
        X=X.to(device)
        Y=Y.to(device)
        
        Y=Y.to(torch.float32)
        
        pred=model(X)

        loss=loss_fn(pred,Y)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        loss_val+=loss.item()
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
        
    print("avg loss=",loss_val/len(dataloader))
    print("avg accuracy=",accruracy_val/len(dataloader))
    


In [33]:
countOfEpoch=15
for epoch in range(countOfEpoch):
    print("Epoch#",epoch+1)
    trainModel(trainDataloader,model,loss_func,optimiser)

Epoch# 1


100%|██████████| 20/20 [00:00<00:00, 216.86it/s]


avg loss= 1.04968381524086
avg accuracy= 0.48
Epoch# 2


100%|██████████| 20/20 [00:00<00:00, 227.28it/s]


avg loss= 0.8477227807044982
avg accuracy= 0.7700000000000001
Epoch# 3


...

avg loss= 0.21147225787863136
avg accuracy= 0.9400000000000001
Epoch# 14


100%|██████████| 20/20 [00:00<00:00, 521.19it/s]


avg loss= 0.19857231611385942
avg accuracy= 0.9400000000000001
Epoch# 15


100%|██████████| 20/20 [00:00<00:00, 505.81it/s]

avg loss= 0.18696075780317187
avg accuracy= 0.9400000000000001


In [34]:

def testModel(dataloader,model):
    accruracy_val=0
    for sample in dataloader:
        X,Y=sample
        
        X=X.to(device)
        Y=Y.to(device)
        
        X=X.to(torch.float32)
        pred=model(X)
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
    print("accuracy=",accruracy_val/len(dataloader))

In [35]:
testDataset=MyDataset(testData)
testDataloader=torch.utils.data.DataLoader(testDataset,batch_size=1)
testModel(testDataloader,model)

accuracy= 0.94
